# Derin Pekiştirmeli Öğrenme (Deep Reinforcement Learning)

In [1]:
import gym #reinforcement için env sağlar , geliştirme ortamı sağlar
import numpy as np
from collections import deque # Ajanın belleğini tanımlamak için deque veri yapısı
from tensorflow.keras.models import Sequential #Sıralı model oluşturmak için 
from tensorflow.keras.layers import Dense  # Tam bağlı (Dense) katamanları
from tensorflow.keras.optimizers import Adam

import random

from tqdm import tqdm #İlerlemeyi görselleştirmek için 

### 1)DQL Agent Class

In [2]:
class DQLAgent:

    # Parametreleri ve hiper parametreleri tanımlama
    def __init__(self,env):
        
        # Çevrenin gözlem alanı (state) boyut sayısı
        self.state_size = env.observation_space.shape[0]

        # Çevrede bulunan eylem sayısı
        self.action_size = env.action_space.n

        # Gelecekteki odüllerin indirim oranı
        self.gamma = 0.95

        #Learning rate : Ajanın öğrenme hızı
        self.learning_rate = 0.001

        # Keşfetme oranı (epsilon) epsilon = 1 olsun max. keşif 
        self.epsilon = 1

        # Epsilon her iterasyonda azalma oranı (epsilon azaldıkça daha fazla öğrenme , daha az keşif)
        self.epsilon_decay = 0.995

        # Min. keşfetme oranı (epsilon : 0.01'in altına inemez)
        self.epsilon_min = 0.01

        # Ajanın deneyimleri = Bellek = Geçmiş adımlar
        self.memory = deque(maxlen = 1000)

        # Derin öğrenme modelini inşaa et
        self.model = self.build_model()

    # DQL sinir ağı modeli oluşturma
    def build_model(self):
        model = Sequential() # Sıralı model

        # Girdi katmanı , 48 nöron , relu aktivasyon
        model.add(Dense(48,input_dim = self.state_size , activation = "relu" ))

        # 24 Nöronlu 2. Gizli katman
        model.add(Dense(24 , activation = "relu" ))

        #Output Katmanı
        model.add(Dense(self.action_size , activation = "linear" ))

        # Modelin Derlenmesi
        model.compile(loss = "mse" , optimizer = Adam(learning_rate = self.learning_rate ) )

        return model
    
    # Ajanın deneyimlerini bellek veri yapısına kaydetme 
    def remember(self , state , action , reward , next_state , done ):
        self.memory.append((state , action , reward , next_state , done)) 

    # Ajanın eylem seçebilme işlemi
    def act(self , state):

        #Eğer rastgele üretilen sayı epsilondan küçükse rastgele eylem seçilir
        if random.uniform(0,1) <= self.epsilon :
            return env.action_space.sample()

        # Aksi durumda model tarafından tahmin edilen değere göre en iyi eylem seçilir 
        act_values = self.model.predict(state , verbose = 0)

        # En yüksek değere sahip eylemi seç
        return np.argmax( act_values[0] )

    # Deneyimleri tekrar oynatarak deep q ağı eğitilir
    def replay(self , batch_size):
        
        # Bellekte yeterinde deneyim yoksa geri oynatma yapılmaz
        if len(self.memory) < batch_size :
            return

        # Bellekte rastgele batch size kadar deneyim seç
        minibatch = random.sample(self.memory , batch_size )

        for state , action , reward , next_state , done in minibatch :
            
            # Eğer done ise bitiş durumu var ise ödülü doğrudan hedef olarak alırız .
            if done:
                target = reward
            else:
                target = reward + self.gamma * np.amax(self.model.predict(next_state , verbose = 0)[0])

            # Modelin tahmin ettiği ödüller 
            train_target = self.model.predict(state , verbose = 0)

            # Ajanın yaptığı eyleme göre ödülü güncelle
            train_target[0][action] = target

            # Modeli eğit
            self.model.fit(state , train_target , verbose = 0)
    # Epsilonun zamanla azalmasını yani keşif ve sömürü dengesi
    def adaptiveEGreedy(self):
        
        if self.epsilon > self.epsilon_min :
            self.epsilon = self.epsilon * self.epsilon_decay

### 2)Model Eğitimi

In [3]:
env = gym.make("CartPole-v1" , render_mode = "human") # Carpole ortamını başlatıyoruz
agent = DQLAgent(env)

batch_size = 32 # Eğitim için minibatch boyutu
episodes = 2# Epochs , simülasyonun oynatılacağı bölüm sayısı

for e in tqdm(range(episodes)):

    # Ortamı sıfırla başlangıç durumunu al
    state = env.reset()[0] # Ortamı sıfırlamak
    state = np.reshape(state ,[1,4])

    time = 0 # Zaman dilimini başlat

    while True :
        # Ajan Eylem seçer 
        action = agent.act(state)

        # Ajan ortamda bir eylem uygular ve bu eylem sonucunda next_state , reward , done (bitiş bilgisi) alır.
        (next_state , reward , done , _ , _ ) = env.step(action)
        next_state = np.reshape(state,[1,4])

        # Yapmış olduğu bu adımı yani eylemi ve bu eylem sonucu env den alınan bilgileri kaydeder
        agent.remember(state , action , reward , next_state , done)

        #Mevcut durumu günceller
        state = next_state 

        # Deneyimlerden yeniden oynatmayı başlatır
        agent.replay(batch_size)

        # Epsilonu set eder 
        agent.adaptiveEGreedy()

        # Zaman dilimini arttırır
        time = time + 1

        # Eğer done ise döngüyü kırar ve bölüm biter ve yeni bölüm başlar
        if done :
            print(f"\nEpisodes : {episodes} Time : {time}")
            break

C:\Users\emirh\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
  0%|                                                                                            | 0/2 [00:00<?, ?it/s]C:\Users\emirh\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
 50%|██████████████████████████████████████████                                          | 1/2 [00:00<00:00,  1.78it/s]


Episodes : 2 Time : 18


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:29<00:00, 14.57s/it]


Episodes : 2 Time : 17


### 3)Modelin Test Edilmesi

In [4]:
import time

trained_model = agent # Eğitilen modeli al
env = gym.make("CartPole-v1",render_mode = "human") #Cartpole ortamını başlatıyoruz
state = env.reset()[0]
state = np.reshape(state , [1,4])

time_t = 0 # Zaman adımını başlat

while True:
    # Ortamı Görsel olarak render et
    env.render() 

    # Eğitilen modelde action gerçekleştir
    action = trained_model.act(state)

    #Eylemi Uygula
    (next_state , reward , done , _ , _ ) = env.step(action)

    next_state = np.reshape(next_state,[1,4])
    state = next_state # Durumun güncellenmesi

    time_t = time_t + 1
    print(f"Time : {time_t}")

    time.sleep(0.5) # 0.5 saniye bekle

    if done :
        break

print("DONE")

Time : 1
Time : 2
Time : 3
Time : 4
Time : 5
Time : 6
Time : 7
Time : 8
Time : 9
Time : 10
Time : 11
Time : 12
Time : 13
Time : 14
Time : 15
Time : 16
Time : 17
Time : 18
DONE
